In [28]:
import matplotlib.pyplot as plt
%matplotlib inline 
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
#from sklearn import cross_validation, linear_model
from sklearn.model_selection import cross_validate
from sklearn.cluster import KMeans
from sklearn import datasets
import random
import time
from numpy.linalg import svd
from musicrecolib.dataset.load_data import LoadData

In [3]:
df = LoadData().get_users_songs()
df.head()

,user_id,song_id,listen_count,track_id,artist_name,track_name,track_uri,popularity,duration_ms,explicit,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,artist_id
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBSUJE12A6D4F8CF5,2,TRPLAXZ128F4292406,Jorge Drexler,12 segundos de oscuridad,2ECKXkpPAxky87ohawpaeD,37,246826,0,...,-8.176,0,0.0327,0.11900,0.000412,0.103,0.0396,126.051,4,4ssUf5gLb1GBLxi1BhPrVt
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBXHDL12A81C204C0,1,TRHNCIR128F42334A5,Kanye West,Stronger,4fzsfWzRhPawzqhX8Qt9F3,82,311867,1,...,-7.858,0,0.1530,0.00564,0.000000,0.408,0.4900,103.992,4,5K4W6rqBFWDnAN6FQUkS6x
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBXHDL12A81C204C0,1,TRUATNS128F423457D,Kanye West,Stronger,4fzsfWzRhPawzqhX8Qt9F3,82,311867,1,...,-7.858,0,0.1530,0.00564,0.000000,0.408,0.4900,103.992,4,5K4W6rqBFWDnAN6FQUkS6x
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBYHAJ12A6701BF1D,1,TRYBNIB128F428E704,Jack Johnson,Constellations,3deZQXBY8CJFbrTc2PbU34,59,201640,0,...,-12.640,1,0.0355,0.46800,0.000043,0.117,0.4430,122.012,4,3GBPw9NK25X1Wt2OUvOwY3
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOEWFWM12A8C1308BA,1,TRLQPQJ128F42AA94F,Gipsy Kings,Soy,076jKe7yfP979o1QLKMIA2,47,189987,0,...,-12.321,0,0.0653,0.29700,0.000267,0.127,0.9050,114.656,4,3jc496ljiyrS3ECrD7QiqL


In [4]:
#adding ratings column to the dataset based on the listen_count

# calculate the maximum listen count
max_listen_count = df['listen_count'].max()

# add a ratings column based on the listen count
df['ratings'] = df['listen_count'] / max_listen_count * 5

                                           user_id             song_id  \
0         b80344d063b5ccb3212f76538f3d9e43d87dca9e  SOBSUJE12A6D4F8CF5   
1         b80344d063b5ccb3212f76538f3d9e43d87dca9e  SOBXHDL12A81C204C0   
2         b80344d063b5ccb3212f76538f3d9e43d87dca9e  SOBXHDL12A81C204C0   
3         b80344d063b5ccb3212f76538f3d9e43d87dca9e  SOBYHAJ12A6701BF1D   
4         b80344d063b5ccb3212f76538f3d9e43d87dca9e  SOEWFWM12A8C1308BA   
...                                            ...                 ...   
11793643  b7815dbb206eb2831ce0fe040d0aa537e2e800f7  SOPUELG12A6701D215   
11793644  b7815dbb206eb2831ce0fe040d0aa537e2e800f7  SORFHOZ12A6701E129   
11793645  b7815dbb206eb2831ce0fe040d0aa537e2e800f7  SOSCPOI12A8C139F02   
11793646  b7815dbb206eb2831ce0fe040d0aa537e2e800f7  SOTIXTZ12AF72A39AC   
11793647  b7815dbb206eb2831ce0fe040d0aa537e2e800f7  SOUJVIT12A8C1451C1   

          listen_count            track_id      artist_name  \
0                    2  TRPLAXZ128F4292406    Jo

In [7]:
# Look for the users who listened to more than 50 songs
usercount = df[["listen_count","user_id"]].groupby("user_id").count()
usercount = usercount[usercount["listen_count"] >= 50]
print(usercount.head())

                                          listen_count
user_id                                               
00028f3cff4872bff3e9985cfa32e01a8d54e374            53
0002e94348b2543c6e6ccf408b0160d14064e46f            74
00038cf792e9f9a1cb593dea5779f96195aac68c            62
00043d7bc800ceff4a90459e189eba5d442a1d3d            67
000b8e9dc26644b1851d516888e7a70f80ea34f1            85


In [15]:
# Look for the songs that were listened to by more than 50 users
workcount = df[["listen_count","user_id"]].groupby("listen_count").count()
workcount = workcount[workcount["user_id"] >= 50]
print(workcount.head())

              user_id
listen_count         
1             6656785
2             1930668
3              900538
4              522939
5              494972


In [24]:
# Keep only the popular songs and active users
df = df[df["user_id"].isin(usercount.index) & df["listen_count"].isin(workcount.index)]
df

,user_id,song_id,listen_count,track_id,artist_name,track_name,track_uri,popularity,duration_ms,explicit,...,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,artist_id,ratings
149,5a905f000fc1ff3df7ca807d57edb608863db05d,SOAJJDS12A8C13A3FB,1,TRWZXDS128F427A71B,The Black Keys,I Got Mine,319zpJMCzpz50Uz0PsjlJN,58,239147,0,...,1,0.0406,0.029200,0.508000,0.1730,0.475,157.725,4,7mnBLXK823vNxN3UWB7Gfz,0.002259
150,5a905f000fc1ff3df7ca807d57edb608863db05d,SOAMDXO12A8C131E2F,2,TRKNKEF128F425FD6A,Digitalism,Pogo,67y8hDBlt0p74eeRwKz1KJ,56,226467,0,...,1,0.0447,0.000065,0.000075,0.5260,0.392,136.013,4,2fBURuq7FrlH6z5F92mpOl,0.004519
151,5a905f000fc1ff3df7ca807d57edb608863db05d,SOATNYF12AF72A8D48,1,TRNJLKP128F427CE28,Daft Punk,Aerodynamic,3H3cOQ6LBLSvmcaV7QkZEu,65,212547,0,...,1,0.0524,0.328000,0.861000,0.0727,0.135,122.902,4,4tZwfgrHOc3mvqYlEYSvVi,0.002259
152,5a905f000fc1ff3df7ca807d57edb608863db05d,SOAUIZJ12A6D4FC011,1,TREHOJT128F14A6622,The Knife,Marble House,59mg5dz40MDexOTFHYRBAb,43,318973,0,...,1,0.0349,0.003200,0.000632,0.3250,0.260,179.660,4,7eQZTqEMozBcuSubfu52i4,0.002259
153,5a905f000fc1ff3df7ca807d57edb608863db05d,SOBDMNP12AF72AB1E1,2,TRGASNY128F14696B0,Daft Punk,Indo Silver Club,4jF8GAMRCOenL17cNnBDI0,47,274733,0,...,1,0.1000,0.088800,0.937000,0.0870,0.443,126.163,4,4tZwfgrHOc3mvqYlEYSvVi,0.004519
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11793612,8305c896f42308824da7d4386f4b9ee584281412,SOZDGEW12A8C13E748,4,TRMLBEG128F42BA124,Metallica,One,5IX4TbIR5mMHGE4wiWwKW0,72,447440,0,...,1,0.0617,0.001110,0.085700,0.1240,0.413,102.178,3,2ye2Wgw4gimLv2eAKyk1NB,0.009038
11793613,8305c896f42308824da7d4386f4b9ee584281412,SOZEETS12AC9071BD3,1,TRZYAGJ128F9332CEF,Justin Bieber,Favorite Girl,4nTjkWK590oBDcnIxTr1mI,58,256800,0,...,1,0.0435,0.047200,0.000000,0.1040,0.634,176.034,4,1uNFoZAHBGtllmzznpCI3s,0.002259
11793614,8305c896f42308824da7d4386f4b9ee584281412,SOZKNSB12A8C140F11,1,TRHPSDO12903CE5189,Iyaz,Replay,4E5P1XyAFtrjpiIxkydly4,78,182307,0,...,1,0.0708,0.173000,0.000000,0.1680,0.195,91.031,4,5tKXB9uuebKE34yowVaU3C,0.002259
11793615,8305c896f42308824da7d4386f4b9ee584281412,SOZVCRW12A67ADA0B7,2,TRNTALF128EF343800,The Killers,When You Were Young,70wYA8oYHoMzhRRkARoMhU,77,220427,0,...,1,0.1120,0.000152,0.048400,0.2800,0.321,130.433,4,0C0XlULifJtAgn6ZNCW2eu,0.004519


In [27]:
# we transform the database into a matrix
dfmatrix = df.pivot(index="user_id", columns="track_id", values="ratings").fillna(0)
dfmatrix

track_id,TRAAABD128F429CF47,TRAAFJW128F428A424,TRAAFON128F42926FB,TRAAGAM128F428DCBE,TRAAGCG128F421CC9F,TRAAGCZ128F93210FD,TRAAGNS128E07824BD,TRAAIDR128F145A5C2,TRAAIHL128F92E6DDA,TRAAJYX128F4255568,...,TRZZKWZ128F423223C,TRZZNHL12903CEB05A,TRZZNVZ12903CBFB4D,TRZZOCK128F930FDDD,TRZZRQJ128F1468136,TRZZRTB128F14603DC,TRZZSEL128F92DEF79,TRZZVWJ128F148C6F4,TRZZZYR128F92F0796,TRZZZYV128F92E996D
user_id,,,,,,,,,,,,,,,,,,,,,
00028f3cff4872bff3e9985cfa32e01a8d54e374,0.0,0.0,0.0,0.0,0.0,0.011297,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0002e94348b2543c6e6ccf408b0160d14064e46f,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00038cf792e9f9a1cb593dea5779f96195aac68c,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00043d7bc800ceff4a90459e189eba5d442a1d3d,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
000b8e9dc26644b1851d516888e7a70f80ea34f1,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fff90dcb01432ff9b9b837ed9af3c533d533fa22,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
fffb0b218640d86e5cb99d41cd3ecad977142da5,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
fffb6725f85dd2bf0742af20a03e6f1f71bf25f0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
# we apply SVD
matrix = dfmatrix.values
u, s, vh = svd(matrix, full_matrices=False)

In [30]:
# we use cosine similarity to find the most similar songs to the one we are looking at
def cosine_similarity(v,u):
    return (v @ u)/ (np.linalg.norm(v) * np.linalg.norm(u))
 
highest_similarity = -np.inf
highest_sim_col = -1
for col in range(1,vh.shape[1]):
    similarity = cosine_similarity(vh[:,0], vh[:,col])
    if similarity > highest_similarity:
        highest_similarity = similarity
        highest_sim_col = col
 
print("Column %d is most similar to column 0" % highest_sim_col)

Column 12579 is most similar to column 0


In [32]:
dfmatrix[dfmatrix.columns[::12579]]

track_id,TRAAABD128F429CF47,TRSEPCD128F92E22C6
user_id,,
00028f3cff4872bff3e9985cfa32e01a8d54e374,0.0,0.0
0002e94348b2543c6e6ccf408b0160d14064e46f,0.0,0.0
00038cf792e9f9a1cb593dea5779f96195aac68c,0.0,0.0
00043d7bc800ceff4a90459e189eba5d442a1d3d,0.0,0.0
000b8e9dc26644b1851d516888e7a70f80ea34f1,0.0,0.0
...,...,...
fff90dcb01432ff9b9b837ed9af3c533d533fa22,0.0,0.0
fffb0b218640d86e5cb99d41cd3ecad977142da5,0.0,0.0
fffb6725f85dd2bf0742af20a03e6f1f71bf25f0,0.0,0.0
